## Step 3 Create final network graph and calculate link costs

1. Process network spatial data into a routable network graph format
2. Reconcile networks into one through node and link overlap conflation
3. __Create final network graph and calculate link costs__
4. Create OD tables
5. Run BikewaySim

In [5]:
from pathlib import Path

In [6]:
from prepare_network import *

## Export Filepath

In [ ]:
export_fp = Path.home() / Path('Documents/BikewaySimData/trb2023')

#### Retrieve Relevant Attributes for Specifying Link Costs
In the network attributes folder there are several '.py' files. These were used to format the network file.

## Link Costs Dictionary
Dict keys must correspond to column names in links GeoDataFrame.
The links cost funciton is off this format:
$$ linkcost = linkdistance * (1-\sum \beta x) $$

Negative attributes **decrease** impedance  
Positive attributes **increase** impedance

In [7]:
costs = {
    'bl':-0.05,
    'pbl':-0.25,#0.95,
    'mu':-0.25,
    'below25':0,
    '25-30':0.30,
    'above30':1,
    '1laneper':0,
    '2to3lanesper':0.50,
    '4ormorelanesper':1,
    'wrongway':2       
    }

In [ ]:
#%% TRB 2023 Code bring in links and nodes

fp = Path.home() / Path('Documents/BikewaySimData/processed_shapefiles/conflation/finalized_networks/here_trb.gpkg')
links = gpd.read_file(fp,layer='links')
fp = Path.home() / Path('Documents/BikewaySimData/processed_shapefiles/here/here_bikewaysim_network.gpkg')
nodes = gpd.read_file(fp,layer='roadbike_nodes')

#prepare network
links, nodes = prepare_network(links,nodes,costs,'imp')

#export
nodes.to_file(export_fp/Path('network.gpkg'),layer='nodes',driver='GPKG')
links.to_file(export_fp/Path('network.gpkg'),layer='links',driver='GPKG')

## See effects of link costs

In [ ]:
#where did link costs go up?
links[links['imp'] > links['dist']].to_file('link_cost_changes.gpkg',layer='increased')

#where did they decrease?
links[links['imp'] < links['dist']].to_file('link_cost_changes.gpkg',layer='decreased')

In [ ]:
#make improvements
improvements = links.copy()
changes = '10TH ST NW'
#add mu
improvements.loc[improvements['ST_NAME']==changes,'mu'] = 1

#bring in new feature
new = gpd.read_file('trb2023/network_improvements.gpkg',layer='new')

#make reverse
new = create_reverse_links(new)

#add to network
improvements = improvements.append(new)

#add impr cost
improvements['imp_dist'] = link_costs(improvements)

#export
improvements.to_file('trb2023/network.gpkg',layer='new_links',driver='GPKG')

In [ ]:
#%% NCST Project
road_links = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='road_links')
road_nodes = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='road_nodes')

bike_links = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='bike_links')
bike_nodes = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='bike_nodes')

roadbike_links = road_links.append(bike_links)
roadbike_nodes = road_nodes.append(bike_nodes).drop_duplicates()

roadbike_links.rename(columns={'osm_A':'A','osm_B':'B'},inplace=True)

links, nodes = prepare_network(roadbike_links,roadbike_nodes,link_costs=True)

print('exporting...')
links.to_file(Path.home() / Path('Documents/TransitSimData/Data/osm_network.gpkg'),layer='links',driver='GPKG')
nodes.to_file(Path.home() / Path('Documents/TransitSimData/Data/osm_network.gpkg'),layer='nodes',driver='GPKG')
print('done.')